In [9]:
# Import necessary libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models

# Ensure NLTK data is downloaded (only needs to be run once)
# nltk.download('stopwords')

# Load the dataset
try:
    df = pd.read_csv('cleaned_english_corpus.csv')
    # Assuming the CSV has columns like 'date' and 'text'
    print("Dataset loaded successfully.")
    print(df.head())
except FileNotFoundError:
    print("Error: 'cleaned_english_corpus.csv' not found. Please ensure the file is in the correct directory.")

Dataset loaded successfully.
       filename                                     cleaned_tokens
0  k210121a.pdf  ['januari', 'monetari', 'polici', 'monetari', ...
1  k210319a.pdf  ['march', 'effect', 'sustain', 'monetari', 'ea...
2  k210427a.pdf  ['april', 'monetari', 'polici', 'monetari', 'p...
3  k210618a.pdf  ['june', 'monetari', 'polici', 'monetari', 'po...
4  k210716a.pdf  ['juli', 'monetari', 'polici', 'monetari', 'po...


In [10]:
def preprocess_text(text):
    """
    Cleans and tokenizes a single text document according to the paper's methodology.
    """
    # 1. Convert to lower case
    text = text.lower()
    # 2. Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    # 3. Tokenize into words
    tokens = text.split()
    # 4. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # 5. Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

# Apply preprocessing to the text column
# Ensure your text column is named 'text' or change accordingly
processed_docs = df['cleaned_tokens'].apply(preprocess_text)

# Create the two main inputs for the LDA model
# 1. Dictionary: A mapping of each unique token to an integer ID
dictionary = corpora.Dictionary(processed_docs)
# Optional: Filter out extreme values (e.g., words in < 5 docs or > 50% of docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5)

# 2. Bag-of-Words (BoW) Corpus: Document representation as (token_id, count)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

print(f"Vocabulary Size (V): {len(dictionary)}")
print(f"Number of Documents: {len(bow_corpus)}")

Vocabulary Size (V): 3581
Number of Documents: 39


In [11]:
def train_lda_model(corpus, dictionary, num_topics):
    """
    Trains a Gensim LDA model with specified hyperparameters.
    
    Args:
        corpus (list): The Bag-of-Words corpus.
        dictionary (corpora.Dictionary): The corpus dictionary.
        num_topics (int): The number of topics (K).
        
    Returns:
        gensim.models.LdaModel: The trained LDA model.
    """
    # Calculate hyperparameters based on the paper's specification
    V = len(dictionary)
    alpha = 50 / num_topics
    eta = 200 / V
    
    print(f"Training LDA with K={num_topics}, alpha={alpha:.4f}, eta={eta:.4f}")
    
    # Train the LDA model
    lda_model = models.LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        alpha=alpha,
        eta=eta,
        passes=15,          # Number of passes through the corpus
        iterations=400,     # Maximum number of iterations
        random_state=42     # For reproducibility
    )
    
    return lda_model

# Example: Train a model with K=10 topics
K = 10
lda_model = train_lda_model(bow_corpus, dictionary, num_topics=K)

Training LDA with K=10, alpha=5.0000, eta=0.0559


In [14]:
import pandas as pd

def extract_topic_probabilities(lda_model, corpus, num_topics):
    """
    Extracts topic probabilities for each document and formats them into a DataFrame.
    """
    # FIX 1: Initialize an empty list
    topic_distributions = []
    
    for doc_bow in corpus:
        # Get the topic distribution for the document
        doc_topics = lda_model.get_document_topics(doc_bow, minimum_probability=0)
        
        # Create a dense vector of probabilities, ensuring it matches the number of topics
        topic_prob_dict = dict(doc_topics)
        topic_prob_vector = [topic_prob_dict.get(i, 0.0) for i in range(num_topics)]
        
        topic_distributions.append(topic_prob_vector)
        
    # FIX 2: Create a list of column names for the DataFrame
    column_names = [f'Topic_{i+1}' for i in range(num_topics)]
    topic_df = pd.DataFrame(topic_distributions, columns=column_names)
    
    return topic_df

# Extract the probabilities
topic_prob_df = extract_topic_probabilities(lda_model, bow_corpus, num_topics=K)

# Combine with original DataFrame (e.g., with dates)
# Assuming df has a 'date' column and its length matches the corpus
final_df = pd.concat([df[['filename']].reset_index(drop=True), topic_prob_df], axis=1)

# FIX 3: Prepare the final matrix of K-1 exogenous variables for the VAR model
# We drop the date and one topic (e.g., 'Topic_1') to use as a baseline
X_t = final_df.drop(columns=['filename', 'Topic_1'])

print("Final DataFrame of textual factors (X_t):")
print(X_t.head())

Final DataFrame of textual factors (X_t):
    Topic_2   Topic_3   Topic_4   Topic_5   Topic_6   Topic_7   Topic_8  \
0  0.015211  0.020688  0.024482  0.012049  0.770797  0.009857  0.009769   
1  0.006348  0.009155  0.503089  0.002517  0.027252  0.002455  0.002443   
2  0.018578  0.019675  0.021411  0.011879  0.834146  0.011695  0.011582   
3  0.017454  0.019884  0.013560  0.006455  0.250214  0.006327  0.006288   
4  0.013608  0.030040  0.032961  0.010234  0.489727  0.008400  0.008344   

    Topic_9  Topic_10  
0  0.017413  0.009971  
1  0.005590  0.002485  
2  0.017802  0.011858  
3  0.013811  0.006405  
4  0.019305  0.008513  


In [15]:
# 1. Print top words for each topic
print("\nTop words for each topic:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

# 2. Calculate topic coherence
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print(f'\nCoherence Score (C_v): {coherence_lda:.4f}')


Top words for each topic:
Topic: 0 
Words: 0.016*"'purchas'," + 0.012*"'financi'," + 0.012*"'amount'," + 0.010*"'price'," + 0.010*"'yen'," + 0.010*"'polici'," + 0.009*"'covid'," + 0.009*"'jgb'," + 0.009*"'interest'," + 0.008*"'increas',"

Topic: 1 
Words: 0.045*"'price'," + 0.015*"'econom'," + 0.014*"'polici'," + 0.013*"'develop'," + 0.013*"'increas'," + 0.013*"'rise'," + 0.012*"'inflat'," + 0.012*"'cpi'," + 0.011*"'wage'," + 0.011*"'moder',"

Topic: 2 
Words: 0.021*"'monetari'," + 0.020*"'polici'," + 0.018*"'price'," + 0.015*"'interest'," + 0.012*"'effect'," + 0.010*"'review'," + 0.009*"'rate'," + 0.009*"'chart'," + 0.009*"'financi'," + 0.009*"'eas',"

Topic: 3 
Words: 0.028*"'interest'," + 0.020*"'financi'," + 0.019*"'yield'," + 0.019*"'rate'," + 0.015*"'price'," + 0.014*"'curv'," + 0.013*"'control'," + 0.013*"'effect'," + 0.012*"'monetari'," + 0.012*"'purchas',"

Topic: 4 
Words: 0.010*"'monetari'," + 0.010*"'polici'," + 0.010*"'price'," + 0.007*"'increas'," + 0.006*"'financi'," + 

In [17]:
# =============================================================================
#  VAR and VAR-teXt Implementation for Macroeconomic Forecasting
# =============================================================================
#
#  Description: This script implements a baseline Vector Autoregression (VAR)
#               model and a text-augmented VAR (VAR-teXt) using the 'statsmodels'
#               library. It is designed to integrate the textual factors (X_t)
#               generated from an LDA model as exogenous variables.
#
#  Required Libraries:
#  pip install pandas numpy statsmodels matplotlib
#
# =============================================================================

import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

# --- 1. Data Preparation ---
# This section simulates macroeconomic data and the textual factors (X_t).
#!!! IMPORTANT: REPLACE THIS SECTION WITH YOUR ACTUAL DATA LOADING!!!

# Simulate macroeconomic data (endogenous variables)
np.random.seed(42)
dates = pd.to_datetime(pd.date_range(start='2010-01-01', periods=150, freq='MS'))
macro_data = pd.DataFrame(np.random.randn(150, 3), columns=['inflation', 'gdp_growth', 'interest_rate'], index=dates)
macro_data = macro_data.cumsum() # Create some trend to make it more realistic
print("--- Simulated Macroeconomic Data (Endogenous) ---")
print(macro_data.head())

# Simulate the textual factors DataFrame (X_t)
# This should have the same index as your macro data.
num_topics = 10
# The textual factors are the K-1 topic probabilities.
exog_cols = []
textual_factors_Xt = pd.DataFrame(np.random.rand(150, num_topics - 1), columns=exog_cols, index=dates)
print("\n--- Simulated Textual Factors (Exogenous) ---")
print(textual_factors_Xt.head())

# --- 2. Pre-Modeling Analysis: Stationarity ---
# VAR models assume the time series are stationary. A common way to achieve this
# is by differencing the data. We use the Augmented Dickey-Fuller test to check.

def check_stationarity(data):
    """Performs ADF test and prints results."""
    for col in data.columns:
        result = adfuller(data[col])
        print(f'ADF Test for {col}:')
        print(f'  ADF Statistic: {result}')
        print(f'  p-value: {result[1]}')
        if result[1] > 0.05:
            print(f'  Result: {col} is likely non-stationary (p > 0.05)\n')
        else:
            print(f'  Result: {col} is likely stationary (p <= 0.05)\n')

print("\n--- Stationarity Check on Original Data ---")
check_stationarity(macro_data)

# If data is non-stationary, apply differencing
macro_data_stationary = macro_data.diff().dropna()
print("\n--- Stationarity Check on Differenced Data ---")
check_stationarity(macro_data_stationary)


# --- 3. Align Data and Split into Training/Testing Sets ---

# Align the exogenous (text) data with the stationary endogenous (macro) data.
# The VAR-teXt model uses lagged textual factors (X_{t-1}) to predict Y_t.
# We shift the textual factors by one period.
textual_factors_lagged = textual_factors_Xt.shift(1)

# Combine and drop any rows with NaN values resulting from differencing and shifting
combined_data = pd.concat([macro_data_stationary, textual_factors_lagged], axis=1).dropna()

# Separate back into endogenous and exogenous DataFrames
endog_data = combined_data[['inflation', 'gdp_growth', 'interest_rate']]
exog_data = combined_data[exog_cols]

# Define the test set size (e.g., last 24 months for forecasting)
n_forecast = 24
endog_train = endog_data.iloc[:-n_forecast]
endog_test = endog_data.iloc[-n_forecast:]
exog_train = exog_data.iloc[:-n_forecast]
exog_test = exog_data.iloc[-n_forecast:]

print(f"\nTraining data shape (Endogenous): {endog_train.shape}")
print(f"Test data shape (Endogenous):     {endog_test.shape}")
print(f"Training data shape (Exogenous): {exog_train.shape}")
print(f"Test data shape (Exogenous):     {exog_test.shape}")


# --- 4. Baseline VAR Model (without textual factors) ---

print("\n--- Fitting Baseline VAR Model ---")
# Instantiate the VAR model on the endogenous training data
model_baseline = VAR(endog_train)

# Select the optimal lag order (p) using AIC
# This is a crucial step for VAR model specification
lag_selection_results = model_baseline.select_order(maxlags=10)
selected_lags = lag_selection_results.aic
print(f"Optimal lag order selected by AIC: {selected_lags}")

# Fit the model with the selected lag order
results_baseline = model_baseline.fit(selected_lags)
print("\nBaseline VAR Model Summary:")
print(results_baseline.summary())

# Generate forecasts
# The number of initial observations must match the lag order
lag_order_baseline = results_baseline.k_ar
forecast_input_baseline = endog_data.values[-n_forecast - lag_order_baseline:-n_forecast]
forecast_baseline = results_baseline.forecast(y=forecast_input_baseline, steps=n_forecast)

# Create a DataFrame for the forecasts
forecast_df_baseline = pd.DataFrame(forecast_baseline, index=endog_test.index, columns=[f'{col}_pred' for col in endog_test.columns])


# --- 5. VAR-teXt Model (with textual factors) ---

print("\n--- Fitting VAR-teXt Model ---")
# Instantiate the VAR model with both endogenous and exogenous training data
model_text = VAR(endog=endog_train, exog=exog_train)

# Fit the model using the same lag order for comparability
results_text = model_text.fit(selected_lags)
print("\nVAR-teXt Model Summary:")
print(results_text.summary())

# Generate forecasts
# The forecast method requires future values of the exogenous variables
lag_order_text = results_text.k_ar
forecast_input_text = endog_data.values[-n_forecast - lag_order_text:-n_forecast]
forecast_text = results_text.forecast(y=forecast_input_text, steps=n_forecast, exog_future=exog_test.values)

# Create a DataFrame for the forecasts
forecast_df_text = pd.DataFrame(forecast_text, index=endog_test.index, columns=[f'{col}_pred' for col in endog_test.columns])


# --- 6. Evaluate and Visualize Forecasts ---

print("\n--- Forecast Evaluation ---")
# Combine actuals and forecasts for comparison
results_df_baseline = pd.concat([endog_test, forecast_df_baseline], axis=1)
results_df_text = pd.concat([endog_test, forecast_df_text], axis=1)

# Calculate Root Mean Squared Error (RMSE) for each model
rmse_baseline = np.sqrt(np.mean((results_df_baseline['inflation'] - results_df_baseline['inflation_pred'])**2))
rmse_text = np.sqrt(np.mean((results_df_text['inflation'] - results_df_text['inflation_pred'])**2))

print(f"RMSE for Baseline VAR (Inflation): {rmse_baseline:.4f}")
print(f"RMSE for VAR-teXt (Inflation):     {rmse_text:.4f}")

if rmse_text < rmse_baseline:
    print("\nThe VAR-teXt model shows improved forecasting performance based on RMSE.")
else:
    print("\nThe VAR-teXt model did not improve forecasting performance based on RMSE.")

# Plot the results for one variable (e.g., inflation)
plt.figure(figsize=(12, 6))
plt.plot(endog_data.index, endog_data['inflation'], label='Actual Inflation', color='black')
plt.plot(results_df_baseline.index, results_df_baseline['inflation_pred'], label='Baseline VAR Forecast', linestyle='--', color='blue')
plt.plot(results_df_text.index, results_df_text['inflation_pred'], label='VAR-teXt Forecast', linestyle='--', color='red')
plt.title('Inflation Forecast: Baseline VAR vs. VAR-teXt')
plt.xlabel('Date')
plt.ylabel('Inflation (Differenced)')
plt.legend()
plt.grid(True)
plt.show()

--- Simulated Macroeconomic Data (Endogenous) ---
            inflation  gdp_growth  interest_rate
2010-01-01   0.496714   -0.138264       0.647689
2010-02-01   2.019744   -0.372418       0.413552
2010-03-01   3.598957    0.395017      -0.055923
2010-04-01   4.141517   -0.068401      -0.521653
2010-05-01   4.383479   -1.981681      -2.246570


ValueError: Shape of passed values is (150, 9), indices imply (150, 0)